<a href="https://colab.research.google.com/github/gustavsma/MD/blob/main/Unbiased_Teacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unbiased Teacher for custom dataset


This notebook is intened as an example for custom dateset with no annotations for the unlabelled images.


**Instructions**

1.   Register dataset.

To do so, add the following code to train_net.py:

```
from detectron2.data.datasets import register_coco_instances

register_coco_instances("***YOR DATASET NAME***", {}, "***PATH TO ANNOTATION JSON FILE***", "***PATH TO IMG FOLDER***")
```

Do so for both training and validation datasets. Training dataset should include both labeled and unlabeled images.

Replace this code in `unbiased-teacher/ubteacher/data/build.py`:
```
# read from pre-generated data seed
    with open(random_data_seed_path) as COCO_sup_file:
        coco_random_idx = json.load(COCO_sup_file)

labeled_idx = np.array(coco_random_idx[str(SupPercent)][str(random_data_seed)])
```
With:
`labeled_idx = np.array([i for i in range(1, num_label+1)])`

Please note that in order for this code to work labelled images must come first in your custom dataset. If you do not know how to do that:

Load both labelled and unlabelled images in an annotation softer that can export COCO format annotations (e.g CVAT). Make sure that labelled images are loaded first. Export tham in COCO format.

2.   Create a config file with your dataset and number of classes:

```
DATASETS:
  CROSS_DATASET: False
  TRAIN: ("***YOR TRAIN DATASET NAME***",)
  TEST: ("***YOR TEST DATASET NAME***",)
```
```
 ROI_HEADS:
    NAME: "StandardROIHeadsPseudoLab"
    LOSS: "FocalLoss"
    NUM_CLASSES: 1
```
A good idea would be to make a copy of `unbiased-teacher/configs/coco_supervision/faster_rcnn_R_50_FPN_sup1_run1.yaml` and replace it with your required parameters. The ones mentioned above are the ones that are required, feel fre to change others aswell.



## 0. Setup

In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
# %cd /content/gdrive/MyDrive
# !git clone https://github.com/facebookresearch/unbiased-teacher

# %cd /content/gdrive/MyDrive/unbiased-teacher
# !mkdir datasets

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# %cd /content/gdrive/MyDrive/unbiased-teacher/datasets
# !ln -s /content/gdrive/MyDrive/data/coco weeds

## 1. Training

**BASE:**

In [ ]:
%%bash
cd /content/gdrive/MyDrive/unbiased-teacher
python train_net.py \
      --num-gpus 1 \
      --config configs/coco_supervision/weeds_base.yaml

**SSL:**

In [ ]:
%%bash
cd /content/gdrive/MyDrive/unbiased-teacher
python train_net.py \
      --num-gpus 1 \
      --config configs/coco_supervision/weeds_ssl.yaml \
      DATALOADER.SUP_PERCENT 9.1

**RESUME:**

In [ ]:
%%bash
cd /content/gdrive/MyDrive/unbiased-teacher
python train_net.py \
      --resume \
      --num-gpus 1 \
      --config configs/coco_supervision/weeds_ssl.yaml \
       DATALOADER.SUP_PERCENT 9.1 MODEL.WEIGHTS /content/gdrive/MyDrive/unbiased-teacher/output/model_final.pth

**ZIP OUTPUT:**

In [ ]:
!zip -r /content/gdrive/MyDrive/x10.zip /content/gdrive/MyDrive/unbiased-teacher/output

## 2. Eval

In [ ]:
%%bash
cd /content/gdrive/MyDrive/unbiased-teacher
python train_net.py \
      --eval-only \
      --num-gpus 1 \
      --config configs/coco_supervision/weeds_ssl.yaml \
      MODEL.WEIGHTS output/model_best.pth

## 3. Inference

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("weeds_train_base", {}, "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/annotations/instances_train_base.json", "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/train_base")
register_coco_instances("weeds_train_x1", {}, "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/annotations/instances_train_x1.json", "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/train_x1")
register_coco_instances("weeds_train_x2", {}, "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/annotations/instances_train_x2.json", "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/train_x2")
register_coco_instances("weeds_train_x5", {}, "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/annotations/instances_train_x5.json", "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/train_x5")
register_coco_instances("weeds_train_x10", {}, "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/annotations/instances_train_x10.json", "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/train_x10")
register_coco_instances("weeds_val", {}, "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/annotations/instances_val.json", "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/val")
register_coco_instances("weeds_test", {}, "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/annotations/instances_test.json", "/content/gdrive/MyDrive/unbiased-teacher/datasets/weeds/test")

In [ ]:
from detectron2.config import get_cfg

cfg = get_cfg()

cfg.merge_from_file("/content/gdrive/MyDrive/unbiased-teacher/configs/Base-RCNN-FPN.yaml")
cfg.MODEL.WEIGHTS = os.path.join("/content/gdrive/MyDrive/unbiased-teacher/output/model_best.pth")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.70
cfg.DATASETS.TRAIN = ("weeds_train_x10", )
cfg.DATASETS.TEST = ("weeds_test", )

In [ ]:
%cd /content/gdrive/MyDrive/unbiased-teacher

from detectron2.data import DatasetCatalog
import cv2, random
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import Visualizer
from ubteacher.engine.trainer import CustomPredictor


dataset_dicts = DatasetCatalog.get("weeds_test")
predictor = CustomPredictor(cfg)

for count, d in enumerate(dataset_dicts[:2]):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1],
                   scale=1
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    img = out.get_image()

    for ann in d['annotations']:
      c = ann['bbox']

      xmin = int(c[0] + c[2])
      ymin = int(c[1] + c[3])
      xmax = int(c[0])
      ymax = int(c[1])

      img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)

    cv2_imshow(img[:, :, ::-1])

    # path = '/content/imgs'
    # cv2.imwrite(os.path.join(path , str(count) + '.jpg'), out.get_image()[:, :, ::-1])
    # cv2.waitKey(0)